In [1]:
from IPython.display import Markdown
from IPython.core.magic import register_cell_magic

from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

import pandas as pd


In [2]:
parks = pd.read_csv("parks2.csv",names=['id','parkName']).drop_duplicates()


In [3]:
img_lst = []
for parkname in parks['parkName'].unique():
    
    park_img_lst = [parkname]
  
    target = '_'.join(parkname.split(' '))
    try:
        html = urlopen('https://en.wikipedia.org/wiki/%s'%target)
        bs = BeautifulSoup(html, 'html.parser')
    except:
        print('park link failed to parse:  https://en.wikipedia.org/wiki/%s'%target )
        continue
    images = bs.find_all('img', {'src':re.compile('.jpg')})
    cntr = 0
    for image in images:
        if "map" not in image["src"]:
            cntr+=1
            park_img_lst.append(str(image))
            park_img_lst.append("https:"+str(image['src']))
            park_img_lst.append("Wikimedia Commons")
        if cntr==2: # got two pictures. done
            img_lst.append(park_img_lst)
            break;

        

park link failed to parse:  https://en.wikipedia.org/wiki/Wrangell–St._Elias_National_Park


In [4]:
#  edge case:
html0 = '<a title="Mr. David Sinson, NOAA, Office of Coast Survey, Public domain, via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:Mt_Saint_Elias.jpg"><img width="512" alt="Mt Saint Elias" src="https://upload.wikimedia.org/wikipedia/commons/0/05/Mt_Saint_Elias.jpg"></a>'
img0 = 'https://upload.wikimedia.org/wikipedia/commons/0/05/Mt_Saint_Elias.jpg'
html1 = '<a title="Ken Thomas (talk contribs), Public domain, via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:St_Mary_Lake.jpg"><img width="512" alt="St Mary Lake" src="https://upload.wikimedia.org/wikipedia/commons/thumb/7/7e/St_Mary_Lake.jpg/512px-St_Mary_Lake.jpg"></a>'
img1 = 'https://upload.wikimedia.org/wikipedia/commons/thumb/7/7e/St_Mary_Lake.jpg/512px-St_Mary_Lake.jpg'

img_lst.append(["Glacier National Park", html0,img0,"Wikimedia Commons",html1,img1,"Wikimedia Commons" ])



In [5]:
# another edge case:
html0 = '<a title="Robert M. Russell, CC BY-SA 4.0 &lt;https://creativecommons.org/licenses/by-sa/4.0&gt;, via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:Mountain_Goat_at_Hidden_Lake.jpg"><img width="512" alt="Mountain Goat at Hidden Lake" src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/51/Mountain_Goat_at_Hidden_Lake.jpg/512px-Mountain_Goat_at_Hidden_Lake.jpg">'
img0 = 'https://upload.wikimedia.org/wikipedia/commons/thumb/5/51/Mountain_Goat_at_Hidden_Lake.jpg/512px-Mountain_Goat_at_Hidden_Lake.jpg'
html1 = '<a title="AlaskaTrekker at English Wikipedia, CC BY-SA 3.0 &lt;http://creativecommons.org/licenses/by-sa/3.0/&gt;, via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:Wrangells1.jpg"><img width="512" alt="Wrangells1" src="https://upload.wikimedia.org/wikipedia/commons/0/00/Wrangells1.jpg"></a>'
img1 =  'https://upload.wikimedia.org/wikipedia/commons/0/00/Wrangells1.jpg'

img_lst.append(["Wrangell–St. Elias National Park", html0,img0,"Wikimedia Commons",html1,img1,"Wikimedia Commons" ])



In [7]:
image_table = pd.DataFrame(img_lst, 
                           columns=['parkName',
                                    'image1html',
                                    'image1url',
                                    'image1credit',
                                    'image2html',
                                    'image2url',
                                    'image2credit'])

In [8]:
image_table.parkName.nunique(), parks.parkName.nunique()

(63, 63)

In [9]:
merged_image_table = image_table.merge(parks, on='parkName')

In [11]:
merged_image_table.head()

,parkName,image1html,image1url,image1credit,image2html,image2url,image2credit,id
0,Acadia National Park,"<img alt=""Bass Harbor Lighthouse b.jpg"" data-f...",https://upload.wikimedia.org/wikipedia/commons...,Wikimedia Commons,"<img alt="""" class=""thumbimage"" data-file-heigh...",https://upload.wikimedia.org/wikipedia/commons...,Wikimedia Commons,1
1,Arches National Park,"<img alt=""Delicate arch sunset.jpg"" data-file-...",https://upload.wikimedia.org/wikipedia/commons...,Wikimedia Commons,"<img alt="""" class=""thumbimage"" data-file-heigh...",https://upload.wikimedia.org/wikipedia/commons...,Wikimedia Commons,2
2,Badlands National Park,"<img alt=""MK00609 Badlands.jpg"" data-file-heig...",https://upload.wikimedia.org/wikipedia/commons...,Wikimedia Commons,"<img alt=""Black Hills"" data-file-height=""3391""...",https://upload.wikimedia.org/wikipedia/commons...,Wikimedia Commons,3
3,Big Bend National Park,"<img alt="""" class=""thumbimage"" data-file-heigh...",https://upload.wikimedia.org/wikipedia/commons...,Wikimedia Commons,"<img alt="""" class=""thumbimage"" data-file-heigh...",https://upload.wikimedia.org/wikipedia/commons...,Wikimedia Commons,4
4,Biscayne National Park,"<img alt=""Sunlight on an underwater coral reef...",https://upload.wikimedia.org/wikipedia/commons...,Wikimedia Commons,"<img alt=""Conch and whelk shells among trees"" ...",https://upload.wikimedia.org/wikipedia/commons...,Wikimedia Commons,5


In [14]:
merged_image_table.to_csv("image_table.csv",index=None)

In [13]:
for it,park in merged_image_table.iterrows():
    print(park['id'], park['parkName'])
    display(Markdown(park['image1html']))
    display(Markdown(park['image2html']))

    print('\n')

1 Acadia National Park


<img alt="Bass Harbor Lighthouse b.jpg" data-file-height="2142" data-file-width="3213" decoding="async" height="189" src="//upload.wikimedia.org/wikipedia/commons/thumb/7/76/Bass_Harbor_Lighthouse_b.jpg/284px-Bass_Harbor_Lighthouse_b.jpg" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/7/76/Bass_Harbor_Lighthouse_b.jpg/426px-Bass_Harbor_Lighthouse_b.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/7/76/Bass_Harbor_Lighthouse_b.jpg/568px-Bass_Harbor_Lighthouse_b.jpg 2x" width="284"/>

<img alt="" class="thumbimage" data-file-height="1836" data-file-width="3264" decoding="async" height="124" src="//upload.wikimedia.org/wikipedia/commons/thumb/a/a3/Acadia_National_Park_Beach.jpg/220px-Acadia_National_Park_Beach.jpg" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/a/a3/Acadia_National_Park_Beach.jpg/330px-Acadia_National_Park_Beach.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/a/a3/Acadia_National_Park_Beach.jpg/440px-Acadia_National_Park_Beach.jpg 2x" width="220"/>



2 Arches National Park


<img alt="Delicate arch sunset.jpg" data-file-height="1944" data-file-width="2592" decoding="async" height="213" src="//upload.wikimedia.org/wikipedia/commons/thumb/f/f0/Delicate_arch_sunset.jpg/284px-Delicate_arch_sunset.jpg" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/f/f0/Delicate_arch_sunset.jpg/426px-Delicate_arch_sunset.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/f/f0/Delicate_arch_sunset.jpg/568px-Delicate_arch_sunset.jpg 2x" width="284"/>

...

<a title="Mr. David Sinson, NOAA, Office of Coast Survey, Public domain, via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:Mt_Saint_Elias.jpg"><img width="512" alt="Mt Saint Elias" src="https://upload.wikimedia.org/wikipedia/commons/0/05/Mt_Saint_Elias.jpg"></a>

<a title="Ken Thomas (talk contribs), Public domain, via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:St_Mary_Lake.jpg"><img width="512" alt="St Mary Lake" src="https://upload.wikimedia.org/wikipedia/commons/thumb/7/7e/St_Mary_Lake.jpg/512px-St_Mary_Lake.jpg"></a>



60 Wrangell–St. Elias National Park


<a title="Robert M. Russell, CC BY-SA 4.0 &lt;https://creativecommons.org/licenses/by-sa/4.0&gt;, via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:Mountain_Goat_at_Hidden_Lake.jpg"><img width="512" alt="Mountain Goat at Hidden Lake" src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/51/Mountain_Goat_at_Hidden_Lake.jpg/512px-Mountain_Goat_at_Hidden_Lake.jpg">

<a title="AlaskaTrekker at English Wikipedia, CC BY-SA 3.0 &lt;http://creativecommons.org/licenses/by-sa/3.0/&gt;, via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:Wrangells1.jpg"><img width="512" alt="Wrangells1" src="https://upload.wikimedia.org/wikipedia/commons/0/00/Wrangells1.jpg"></a>